In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

START = Int(0)
FREE = Int(1) # No modo Free  não existe qualquer força de travagem
STOPPING = Int(2)
BLOCKED = Int(3) # no modo Blocked as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa)
STOPPED = Int(4) # no modo Stopped o veículo está imobilizado.

a = 5 # Forca de atrito
p = 100 #peso
c = 1
V0 = 60 # Velocidade inicial
T = 1 # Segundos maximo q pode ficar tanto no free como no blockd
#A componente contínua  do autómato usa variáveis contínuas $$\,V,v\,$$ para descrever a 
#velocidade do corpo   e a velocidade linear das rodas  ambas em relação so solo

#F força de travagem (Variavel)
#t Força de atrito a o solo (constante)

'''
1.
Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se 
desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas. 

2.
No contacto rodas/solo o atrito é constante porque a força de compressão é o peso; tem-se f = aP a, a constante de
atrito e P o peso. Ambos são fixos e independentes do modo.

3.
No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à 
diferença de velocidades F =  c(V-v) A  constante de proporcionalidade c depende do modo: é elevada no modo 
`Stopping` e baixa nos outros.


4. As  equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`, (V = -F) ^ (v = -aP +F)
e no modo `Blocked` (V = v) ^ (v = -aP)

5.Tanto no modo Blocked  como no modo Free  existe um “timer” que impede que o controlo aí permaneça mais do que 
t segundos.

6. Todos os “switchs” devem ser construídos de  modo a impedir a existência de trajetórias de Zenão.
                                                        
7. No instante inicial assume-se V = v = V0  ,  em que a velocidade V0 é o “input” do problema.( Isto é, sequências 
infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.)

Declararação das variáveis do FOTS correspondente ao veículo em que t é o tempo , m o estado que o veículo se encontra , V a velocidade do corpo em relação ao solo, V velocidade linear das rodas em relação ao solo.

In [ ]:
def declare(i):
    s = {}
    s['t'] = Symbol('t'+str(i),REAL)
    s['m'] = Symbol('m'+str(i),INT)
    s['V'] = Symbol('x'+str(i),REAL)
    s['v'] = Symbol('x'+str(i),REAL)

    return s

O estado inicial do FOTS é o seguinte :
$$
m = \mathsf{START}\wedge v = V0 \wedge V0 = V \wedge t = 0 \wedge V0 >= 0 \\
$$
Temos que começar no Start e definimos V e v com o valor de V0 que é o "input" do problema , e que diz a velocidade inicial do veículo.

In [ ]:
def init(s):
    return And(Equals(s['m'],START),Equals(s['V'],V0),Equals(s['v'],V0),GE(V0,Real(0)),(Equals(s['t'],Real(0)))

As transiçõe "untimed" do fots são as seguintes : 
$$
\begin{array}{c}
m = \mathsf{START} \wedge m' = \mathsf{FREE} \wedge v' = v \wedge V' = V \wedge t' = t \\
\vee\\
m = \mathsf{FREE} \wedge m' = \mathsf{STOPPING} \wedge v' = v \wedge V' = V \wedge t' = t  \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{BLOCKED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{BLOCKED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v != 0 \wedge V != 0 \\
\vee\\
m = \mathsf{BLOCKED} \wedge m' = \mathsf{STOPPED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \wedge V = 0 \\
\end{array}
As transições "timed " -  switches do fots são as seguintes :
$$

In [ ]:
def trans(s,p):
    #Untimed
    #Falta colocar as alterações de velocidade
    #Definir condições para trocar entre as variaveis que vai no timed ................
    #Stopping momento até bloquear definir
    startFree = And(Equals(s['m'],START),Equals(p['m'],FREE), Equals(s['v'],p['v']) ,Equals(s['V'],p['V'])) 
    freeStopping = And(Equals(s['m'],FREE),Equals(p['m'],STOPPING),Equals(s['v'],p['v']) ,Equals(s['V'],p['V']))
    stopppingBlocked = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPING), Equals(s['v'],p['v']),Equals(Int(0),p['v']) ,Equals(s['V'],p['V']))
    blockedFree =  And(Equals(s['m'],BLOCKED),Equals(p['m'],FREE), Equals(s['v'],p['v']) ,Equals(s['V'],p['V'],NOT(Equals(s['V'],Real(0))), NOT(Equals(s['v'],Real(0))))
    blockedStopped = And(Equals(s['m'],BLOCKED),Equals(p['m'],STOPPED), Equals(s['v'],p['v']) ,Equals(s['V'],p['V'],Equals(s['V'],Real(0)), Equals(s['v'],Real(0)))

    
    #DAR PRO FreeFree uma condição pra ele parar de fazer freeFree sem ser a do timer 
    #provavel colocar alguma porcentagem de diferenca de vel q nem no termostato 
    #Timed - Switchs
    freeFree = And(Equals(s['m'],FREE),Equals(p['m'],FREE),Equals(p['m']),s['t'] - p['t'] <= T , GT(p['V'] , Real(0)),GT(p['v'] , Real(0)),GT(p['t'] > s['t'])#Alteracao V e v de acordo com o tempo ) )
    blockedBlocked = And(Equals(s['m'],BLOCKED),Equals(p['m'],BLOCKED),NOT(Equals(p['v'],Real(0))),GT(p['t'] > s['t'])#Alteracao V e v de acordo com o tempo )
    stoppingStopping = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPING),NOT(Equals(p['v'],Real(0)0),Equals(p['m']),s['t'] - p['t'] <= T,GT(p['t'] > s['t']))

    return Or([startFree ,freeStopping,stopppingBlocked ,blockedFree ,blockedStopped,freeFree,blockedBlocked,stoppingStopping])

In [ ]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        # completar
        trace = [declare(i) for i in range(k)]
        
        #semantics
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
    
                        
####
gera_traco(declare,init,trans,5)